<a href="https://colab.research.google.com/github/Pimwipha302004/Pimwipha302004-Pimwipha-229352-StatisticalLeaening-2/blob/main/lab04_naive_bayes_grid_and_random_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [2]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [5]:
#1.
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

param_grid = {
    'nb__alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1
)

grid_search.fit(Xtrain, ytrain)

print("Best alpha (Grid Search):", grid_search.best_params_['nb__alpha'])
print("Best CV f1_macro:", grid_search.best_score_)



Best alpha (Grid Search): 0.01
Best CV f1_macro: 0.8392496686540294


In [7]:
#2.
from sklearn.metrics import f1_score

best_grid_model = grid_search.best_estimator_

y_pred_grid = best_grid_model.predict(Xtest)

f1_grid = f1_score(ytest, y_pred_grid, average='macro')
print("Test f1_macro (Grid Search):", f1_grid)


Test f1_macro (Grid Search): 0.7708784080844321


alpha = 0.01
f1_macro = 0.7708784080844321

In [9]:
#3.
param_dist = {
    'nb__alpha': uniform(0.0001, 10)
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(Xtrain, ytrain)

print("Best alpha (Random Search):", random_search.best_params_['nb__alpha'])
print("Best CV f1_macro:", random_search.best_score_)


Best alpha (Random Search): 0.20594494295802446
Best CV f1_macro: 0.8111179371649779


alpha = 0.20594494295802446
f1_macro > 0.7709 → better